Diego Toribio <br>
Professor Sam Keene <br>
Frequentist Machine Learning <br>
Project 4: Non-Negative Matrix Factorization

In [ ]:
!pip install scikit-surprise

!wget -nc https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -n ml-100k.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357281 sha256=25ddeb63664fee57450a865eb6d6b52e8463d19e2926a697c10e674ce010317e
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
--2024-12-04 14:42:03--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  8.39MB/s    in 0.6s    

2024-12-04 14:42:04 

In [ ]:
import numpy as np
import pandas as pd
from tabulate import tabulate
from sklearn.model_selection import KFold
from surprise import Dataset, Reader, NMF
from surprise.model_selection import GridSearchCV, train_test_split

In [ ]:
class DataLoader:
    def __init__(self, ratings_file, items_file):
        self.ratings_file = ratings_file
        self.items_file = items_file
        self.ratings_df = None
        self.items_df = None

    def load_data(self):
        """Load ratings and movie metadata."""
        ratings_columns = ["user_id", "item_id", "rating", "timestamp"]
        item_columns = [
            "movie_id",
            "movie_title",
            "release_date",
            "video_release_date",
            "IMDb_URL",
            "unknown",
            "Action",
            "Adventure",
            "Animation",
            "Children's",
            "Comedy",
            "Crime",
            "Documentary",
            "Drama",
            "Fantasy",
            "Film-Noir",
            "Horror",
            "Musical",
            "Mystery",
            "Romance",
            "Sci-Fi",
            "Thriller",
            "War",
            "Western",
        ]

        self.ratings_df = pd.read_csv(
            self.ratings_file, sep="\t", names=ratings_columns, encoding='latin-1'
        )
        self.items_df = pd.read_csv(
            self.items_file,
            sep="|",
            encoding="latin-1",
            header=None,
            names=item_columns,
            usecols=["movie_id", "movie_title"],
        )

    def get_ratings_df(self):
        return self.ratings_df

    def get_items_df(self):
        return self.items_df


class MovieRecommender:
    def __init__(self, data_loader):
        self.data_loader = data_loader
        self.ratings_df = None
        self.items_df = None
        self.data = None
        self.best_algo = None
        self.predictions = None
        self.merged_df = None
        self.trainset = None
        self.testset = None

    def load_data(self):
        self.data_loader.load_data()
        self.ratings_df = self.data_loader.get_ratings_df()
        self.items_df = self.data_loader.get_items_df()

        reader = Reader(rating_scale=(1, 5))
        self.data = Dataset.load_from_df(
            self.ratings_df[["user_id", "item_id", "rating"]], reader
        )

    def train_model(self, param_grid):
        grid = GridSearchCV(
            NMF,
            param_grid=param_grid,
            measures=["rmse"],
            cv=2,
            n_jobs=-1,
            joblib_verbose=1,
        )
        grid.fit(self.data)
        print(f"Best RMSE score: {grid.best_score['rmse']}")
        print("Best parameters:", grid.best_params["rmse"])
        self.best_algo = grid.best_estimator["rmse"]

    def generate_predictions(self, test_size=0.2, random_state=42):
        trainset, self.testset = train_test_split(self.data, test_size=test_size, random_state=random_state)
        self.best_algo.fit(trainset)
        self.predictions = self.best_algo.test(self.testset)

        pred_df = pd.DataFrame(
            [
                {
                    "UserID": int(pred.uid),
                    "ItemID": int(pred.iid),
                    "Estimated Rating": round(pred.est, 3),
                    "Actual Rating": pred.r_ui,
                }
                for pred in self.predictions
            ]
        )

        self.merged_df = pd.merge(
            pred_df,
            self.items_df,
            left_on="ItemID",
            right_on="movie_id",
            how="left",
        )
        self.merged_df.drop(["movie_id"], axis=1, inplace=True)

    def generate_recommendations(self, user_id, n=10):
        trainset = self.data.build_full_trainset()
        self.best_algo.fit(trainset)

        anti_testset = trainset.build_anti_testset()

        user_anti_testset = [pair for pair in anti_testset if pair[0] == str(user_id)]

        predictions = self.best_algo.test(user_anti_testset)

        rec_df = pd.DataFrame(
            [
                {
                    "UserID": int(pred.uid),
                    "ItemID": int(pred.iid),
                    "Estimated Rating": round(pred.est, 3),
                }
                for pred in predictions
            ]
        )

        merged_rec_df = pd.merge(
            rec_df,
            self.items_df,
            left_on="ItemID",
            right_on="movie_id",
            how="left",
        )
        merged_rec_df.drop(["movie_id"], axis=1, inplace=True)

        top_n = merged_rec_df.sort_values("Estimated Rating", ascending=False).head(n)
        return top_n[["UserID", "Estimated Rating", "movie_title"]]

    def get_top_n_recommendations_evaluation(self, user_id, n=10):
        if self.merged_df is None:
            raise ValueError(
                "Predictions have not been generated. Call generate_predictions() first."
            )

        user_predictions = self.merged_df[self.merged_df["UserID"] == user_id]
        user_predictions = user_predictions.sort_values("Estimated Rating", ascending=False)
        return user_predictions.head(n)[["UserID", "Actual Rating", "Estimated Rating", "movie_title"]]


config = {
    "n_factors": [40, 50, 60],
    "n_epochs": [8, 10, 12],
    "reg_pu": [0.5, 1.0, 1.5],
    "reg_qi": [0.05, 0.1, 0.15],
}

ratings_file = "ml-100k/u.data"
items_file = "ml-100k/u.item"

data_loader = DataLoader(ratings_file, items_file)
recommender = MovieRecommender(data_loader)

recommender.load_data()
recommender.train_model(config)
recommender.generate_predictions()

# Get top N predictions from the test set
user_id_eval = 13
top_evaluation = recommender.get_top_n_recommendations_evaluation(user_id_eval, n=10)

print(f"\nTop 10 evaluation predictions for user {user_id_eval}:")
print(tabulate(top_evaluation, headers='keys', tablefmt='pretty'))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  1.6min finished


Best RMSE score: 0.9582954189422881
Best parameters: {'n_factors': 60, 'n_epochs': 10, 'reg_pu': 1.5, 'reg_qi': 0.05}

Top 10 evaluation predictions for user 13:
+-------+--------+---------------+------------------+----------------------------------+
|       | UserID | Actual Rating | Estimated Rating |           movie_title            |
+-------+--------+---------------+------------------+----------------------------------+
|  302  |   13   |      5.0      |      4.133       | Shawshank Redemption, The (1994) |
| 1145  |   13   |      5.0      |      4.082       |      Godfather, The (1972)       |
| 2563  |   13   |      2.0      |      3.941       |         Boot, Das (1981)         |
| 3903  |   13   |      4.0      |      3.923       | Manchurian Candidate, The (1962) |
| 13623 |   13   |      5.0      |      3.887       |    Lawrence of Arabia (1962)     |
| 13801 |   13   |      4.0      |      3.879       |     Great Escape, The (1963)     |
| 15106 |   13   |      2.0      |   

## Stretch Goal \#1

Using an explicit feedback dataset, implement the NMF algorithm by hand, tune it via cross validation to select the # of latent dims and regularization parameter.

In [ ]:
class NMF_SGD:
    def __init__(
        self, R, num_factors=10, alpha=0.001, beta=0.02, epochs=20, random_state=None
    ):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.num_factors = num_factors
        self.alpha = alpha
        self.reg_pu = beta
        self.reg_qi = beta
        self.epochs = epochs
        self.random_state = random_state

    def fit(self):
        np.random.seed(self.random_state)
        self.P = np.random.rand(self.num_users, self.num_factors)
        self.Q = np.random.rand(self.num_items, self.num_factors)

        rows, cols = self.R.nonzero()
        ratings = self.R[rows, cols]

        for _ in range(self.epochs):
            predictions = np.sum(self.P[rows, :] * self.Q[cols, :], axis=1)
            errors = ratings - predictions

            dp = -2 * errors[:, np.newaxis] * self.Q[cols, :] + self.reg_pu * self.P[rows, :]
            dq = -2 * errors[:, np.newaxis] * self.P[rows, :] + self.reg_qi * self.Q[cols, :]

            P_grad = np.zeros_like(self.P)
            Q_grad = np.zeros_like(self.Q)

            np.add.at(P_grad, rows, dp)
            np.add.at(Q_grad, cols, dq)

            self.P -= self.alpha * P_grad
            self.Q -= self.alpha * Q_grad

            self.P = np.maximum(self.P, 0)
            self.Q = np.maximum(self.Q, 0)

    def predict_single(self, i, j):
        return np.dot(self.P[i, :], self.Q[j, :].T)

    def predict_all(self):
        return np.dot(self.P, self.Q.T)


def cross_validate_nmf(
    R, num_factors_list, beta_list, alpha=0.001, epochs=20, n_splits=3
):
    best_rmse = float("inf")
    best_params = {"num_factors": None, "beta": None}

    print("Starting cross-validation...")
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    for num_factors in num_factors_list:
        for beta in beta_list:
            rmses = []
            for train_indices, test_indices in kf.split(R):
                R_train = np.copy(R)
                R_test = np.zeros(R.shape)
                for idx in test_indices:
                    R_train[idx, :] = 0
                    R_test[idx, :] = R[idx, :]
                nmf = NMF_SGD(
                    R_train,
                    num_factors=num_factors,
                    alpha=alpha,
                    beta=beta,
                    epochs=epochs,
                )
                nmf.fit()
                predicted = nmf.predict_all()
                xs, ys = R_test.nonzero()
                error = 0
                count = 0
                for x, y in zip(xs, ys):
                    error += (R_test[x, y] - predicted[x, y]) ** 2
                    count += 1
                rmse = np.sqrt(error / count) if count > 0 else float("inf")
                rmses.append(rmse)
            avg_rmse = np.mean(rmses)
            print(f"num_factors: {num_factors}, beta: {beta}, Avg RMSE: {avg_rmse:.4f}")
            if avg_rmse < best_rmse:
                best_rmse = avg_rmse
                best_params = {"num_factors": num_factors, "beta": beta}

    return best_params, best_rmse


class NMFRecommender:
    def __init__(self, R, items_df):
        self.R = R
        self.items_df = items_df
        self.nmf_model = None
        self.R_pred = None
        self.R_pred_df = None
        self.R_df = pd.DataFrame(R)

    def cross_validate(
        self, num_factors_list, beta_list, alpha=0.001, epochs=20, n_splits=3
    ):
        return cross_validate_nmf(
            self.R, num_factors_list, beta_list, alpha, epochs, n_splits
        )

    def train(self, num_factors, beta, alpha=0.001, epochs=20, random_state=None):
        self.nmf_model = NMF_SGD(
            self.R, num_factors=num_factors, beta=beta, alpha=alpha, epochs=epochs, random_state=random_state
        )
        self.nmf_model.fit()
        self.R_pred = self.nmf_model.predict_all()
        self.R_pred_df = pd.DataFrame(
            self.R_pred, index=self.R_df.index, columns=self.R_df.columns
        )

    def get_top_n_recommendations(self, user_id, n=10):
        user_ratings = self.R_df.loc[user_id]
        user_predictions = self.R_pred_df.loc[user_id]
        unrated_items = user_ratings[user_ratings == 0].index
        predicted_ratings = user_predictions[unrated_items]
        top_n_items = predicted_ratings.sort_values(ascending=False).head(n).index
        top_n_titles = self.items_df[self.items_df["movie_id"].isin(top_n_items)]["movie_title"]
        return pd.DataFrame(
            {
                "movie_id": top_n_items,
                "predicted_rating": predicted_ratings[top_n_items].values,
                "movie_title": top_n_titles.values,
            }
        )

    def get_top_n_recommendations_evaluation(self, user_id, n=10):
        user_ratings = self.R_df.loc[user_id]
        user_predictions = self.R_pred_df.loc[user_id]

        rated_items = user_ratings[user_ratings > 0].index
        user_eval_df = pd.DataFrame({
            "UserID": user_id,
            "Actual Rating": user_ratings[rated_items].values,
            "Estimated Rating": user_predictions[rated_items].values,
            "movie_id": rated_items
        })

        user_eval_df["Estimated Rating"] = np.round(user_eval_df["Estimated Rating"], 3)
        user_eval_df["Estimated Rating"] = np.minimum(user_eval_df["Estimated Rating"], 5.0)

        user_eval_df = user_eval_df.merge(self.items_df, how="left", on="movie_id")
        user_eval_df = user_eval_df.sort_values("Estimated Rating", ascending=False).head(n)

        return user_eval_df[["UserID", "Actual Rating", "Estimated Rating", "movie_title"]]


# Load your data
ratings_file = "ml-100k/u.data"
items_file = "ml-100k/u.item"

data_loader = DataLoader(ratings_file, items_file)
data_loader.load_data()
ratings_df = data_loader.get_ratings_df()
items_df = data_loader.get_items_df()

ratings_df["user_id"] -= 1
ratings_df["item_id"] -= 1
items_df["movie_id"] -= 1

num_users = ratings_df["user_id"].nunique()
num_items = ratings_df["item_id"].nunique()
R_df = ratings_df.pivot(index="user_id", columns="item_id", values="rating").fillna(0)
R = R_df.values

nmf_recommender = NMFRecommender(R, items_df)

num_factors_list = [14, 15, 16]
beta_list = [0.01, 0.05, 0.1]

best_params, best_rmse = nmf_recommender.cross_validate(
    num_factors_list, beta_list, alpha=0.001, epochs=20, n_splits=3
)

nmf_recommender.train(
    num_factors=best_params["num_factors"],
    beta=best_params["beta"],
    alpha=0.001,
    epochs=20,
    random_state=13,
)

# Get top N evaluation predictions for user 13
user_id = 13
top_evaluation = nmf_recommender.get_top_n_recommendations_evaluation(user_id, n=10)

print(f"\nBest RMSE: {best_rmse:.4f}")
print(f"Best Parameters: {best_params}")
print(f"\nTop 10 evaluation predictions for user {user_id}:")
print(tabulate(top_evaluation, headers='keys', tablefmt='pretty'))


Starting cross-validation...
num_factors: 14, beta: 0.01, Avg RMSE: 1.6941
num_factors: 14, beta: 0.05, Avg RMSE: 1.5533
num_factors: 14, beta: 0.1, Avg RMSE: 1.4944
num_factors: 15, beta: 0.01, Avg RMSE: 1.5927
num_factors: 15, beta: 0.05, Avg RMSE: 1.6150
num_factors: 15, beta: 0.1, Avg RMSE: 1.5643
num_factors: 16, beta: 0.01, Avg RMSE: 1.5910
num_factors: 16, beta: 0.05, Avg RMSE: 1.6709
num_factors: 16, beta: 0.1, Avg RMSE: 1.6312

Best RMSE: 1.4944
Best Parameters: {'num_factors': 14, 'beta': 0.1}

Top 10 evaluation predictions for user 13:
+----+--------+---------------+------------------+----------------------------------------+
|    | UserID | Actual Rating | Estimated Rating |              movie_title               |
+----+--------+---------------+------------------+----------------------------------------+
| 30 |   13   |      5.0      |       5.0        |     Raiders of the Lost Ark (1981)     |
| 29 |   13   |      4.0      |       5.0        |       Princess Bride, The (1